In [ ]:
# 01_data_cleaning.ipynb
import pandas as pd
import numpy as np

In [ ]:
# Load dataset
data = pd.read_csv("../data/raw/laptop_price.csv", encoding="ISO-8859-1")

In [6]:
# Show the column names
print("Columns in dataset:")
print(data.columns.tolist())

Columns in dataset:
['laptop_ID', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price_euros']


In [ ]:
data.head()